In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Define a simple label encoder function
def encode_column(column):
    if column.dtype == object:
        column = column.astype('category').cat.codes
    return column

# Apply the encoder to each column

# Assuming 'data.csv' is your CSV file
df = pd.read_csv('breast-cancer-wisconsin-data.csv')
df = df.apply(encode_column)
#print(df.head())
# Separate the features and the target variable
#buying,maint,doors,persons,lug_boot,safety low,eval
#"id","diagnosis","radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean","radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se","concavity_se","concave points_se","symmetry_se","fractal_dimension_se","radius_worst","texture_worst","perimeter_worst","area_worst","smoothness_worst","compactness_worst","concavity_worst","concave points_worst","symmetry_worst","fractal_dimension_worst",

# Separate the features and the target variable
X = df[["id","radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean","radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se","concavity_se","concave points_se","symmetry_se","fractal_dimension_se","radius_worst","texture_worst","perimeter_worst","area_worst","smoothness_worst","compactness_worst","concavity_worst","concave points_worst","symmetry_worst","fractal_dimension_worst"]].to_numpy()
#X = df[['buying','maint','doors','persons','lug_boot','safety']].to_numpy()
y = df['diagnosis'].to_numpy()
#print(y)
#print(X)
 #TODO: ommit patient id
# Remove patient id
# Convert the species labels to integers
species_to_int = {species: idx for idx, species in enumerate(np.unique(y))}
y_int = np.array([species_to_int[eval] for eval in y])

# Optionally, convert the integer labels to one-hot encoding
def one_hot_encode(labels, num_classes):
    return np.eye(num_classes)[labels]

num_classes = len(species_to_int)
y_one_hot = one_hot_encode(y_int, num_classes)
print(y_one_hot.shape, num_classes, X.shape)
#print(X)


(569, 2) 2 (569, 31)


In [35]:
X = X[:,1:]
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

In [36]:
import torch
# Convert to PyTorch tensors
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y_int, dtype=torch.int64).to(device)
x_train, x_test, y_train, y_test = train_test_split(x_tensor, y_tensor, test_size=0.2, random_state=1973)

In [37]:
# x_tensor -= x_tensor.mean(dim=0)
# x_tensor /= x_tensor.std(dim=0)
mean = x_train.mean(dim=0)
std = x_train.std(dim=0)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std
# x_train, x_test, y_train, y_test = train_test_split(x_tensor, y_tensor, test_size=0.2, random_state=1973)

In [38]:

num_class = 2
d = X.shape[1]
from nn_model import NeuralNetwork
model = NeuralNetwork(d, 50, num_class).to(device)

model.fit(x_train, y_train, x_test, y_test, epochs=10, learning_rate=0.01)

Epoch [1/10], Loss: 0.6657, Accuracy: 0.8509
Epoch [2/10], Loss: 0.5916, Accuracy: 0.8772
Epoch [3/10], Loss: 0.5075, Accuracy: 0.9123
Epoch [4/10], Loss: 0.4389, Accuracy: 0.9298
Epoch [5/10], Loss: 0.4011, Accuracy: 0.9474
Epoch [6/10], Loss: 0.3816, Accuracy: 0.9474
Epoch [7/10], Loss: 0.3690, Accuracy: 0.9649
Epoch [8/10], Loss: 0.3604, Accuracy: 0.9737
Epoch [9/10], Loss: 0.3531, Accuracy: 0.9912
Epoch [10/10], Loss: 0.3488, Accuracy: 0.9825


NeuralNetwork(
  (fc1): Linear(in_features=30, out_features=50, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=50, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

In [39]:
print("Test")
print(model.predict(x_test))
print("True")
print(y_test)

Test
tensor([0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1], device='cuda:0')
True
tensor([0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1], device='cuda:0')


In [40]:
# Evaluate the model
y_pred = model.predict(x_test)
acc = (y_pred == y_test).float().mean()
print('Accuracy: {:.2f}%'.format(acc * 100))

Accuracy: 98.25%


In [14]:
# Initialize variables
cut_points_list = [torch.nn.Parameter(torch.rand(i)) for i in num_cut]
leaf_score = torch.nn.Parameter(torch.rand(num_leaf, num_class))

# Define loss and optimizer
optimizer = torch.optim.Adam([*cut_points_list, leaf_score], lr=0.1)
loss_fn = torch.nn.CrossEntropyLoss()
loss_function = torch.nn.CrossEntropyLoss()

In [15]:
from pytorch_neural import nn_decision_tree
# Training loop
for i in range(1000):
    optimizer.zero_grad()
    y_pred = nn_decision_tree(x_tensor, cut_points_list, leaf_score, temperature=0.1)
    loss = loss_fn(y_pred, y_tensor)
    loss.backward()
    optimizer.step()

    if i % 200 == 0:
        print(loss.item())

0.684697687625885
0.2382846623659134
0.22305907309055328
0.22055937349796295
0.22198253870010376


In [16]:
# Evaluate
with torch.no_grad():
    y_pred_eval = nn_decision_tree(x_tensor, cut_points_list, leaf_score, temperature=0.1)
    error_rate = 1 - (y_pred_eval.argmax(1) == y_tensor.argmax(1)).float().mean()
    print('error rate %.2f' % error_rate)

error rate 0.09


In [17]:
from sklearn.metrics import accuracy_score

y_pred = nn_decision_tree(x_tensor, cut_points_list, leaf_score, temperature=0.1)
accuracy_score(y_tensor.argmax(1), y_pred.argmax(1))
print(accuracy_score(y_tensor.argmax(1), y_pred.argmax(1)))

0.9121265377855887


In [18]:
sample_x0 = np.repeat(np.linspace(0, np.max(X[:,0]), 100), 100).reshape(-1,1)
sample_x1 = np.tile(np.linspace(0, np.max(X[:,1]), 100).reshape(-1,1), [100,1])
sample_x = np.hstack([sample_x0, sample_x1])
sample_x_tensor = torch.tensor(sample_x, dtype=torch.float32)

In [19]:
import matplotlib.pyplot as plt
with torch.no_grad():
    sample_label = nn_decision_tree(sample_x_tensor, cut_points_list, leaf_score, temperature=0.1).argmax(1)

plt.figure(figsize=(8,8))
plt.scatter(X[:,0], X[:,1], c=np.argmax(y_one_hot, axis=1), marker='o', s=50, cmap='summer', edgecolors='black')
plt.scatter(sample_x0.flatten(), sample_x1.flatten(), c=sample_label.numpy().flatten(), marker='D', s=20, cmap='summer', edgecolors='none', alpha=0.33)
plt.show()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10000x0 and 1x2)